In [4]:
# Load the dataset
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("Ecommerce Analysis").getOrCreate()

df = spark.read.csv('EcommerceDataset_Final2.csv', header=True, inferSchema=True)
df.show()


+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+
|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|CustomerID|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|
+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+
|    614|     Patricia Smith|   Book|       2|   330.71|2024-09-05 00:00:00|       36.96|         1|   Canada|   Premium|202

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
mode_email_domain = df.groupBy('EmailDomain').count().orderBy('count', ascending=False).first()[0]

# Fill null values in the 'EmailDomain' column with the mode
df = df.withColumn('EmailDomain', when(col('EmailDomain').isNull(), mode_email_domain).otherwise(col('EmailDomain')))
df.show()

+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+
|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|CustomerID|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|
+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+
|    614|     Patricia Smith|   Book|       2|   330.71|2024-09-05 00:00:00|       36.96|         1|   Canada|   Premium|202

In [14]:



# Identify unique product visits and map to reference channels
unique_visits = df.groupBy('Product', 'EmailDomain').agg(countDistinct('CustomerID').alias('UniqueVisits')).sort(["Product","UniqueVisits"],ascending=True)
unique_visits.show()


+-------+-------------+------------+
|Product|  EmailDomain|UniqueVisits|
+-------+-------------+------------+
|   Bike|bestdeals.com|          33|
|   Bike|    eshop.com|          39|
|   Bike|  buyeasy.com|          41|
|   Bike|shopworld.com|          44|
|   Book|shopworld.com|          47|
|   Book|bestdeals.com|          47|
|   Book|  buyeasy.com|          59|
|   Book|    eshop.com|          61|
| Laptop|shopworld.com|          36|
| Laptop|bestdeals.com|          42|
| Laptop|  buyeasy.com|          45|
| Laptop|    eshop.com|          48|
|  Shirt|  buyeasy.com|          35|
|  Shirt|shopworld.com|          36|
|  Shirt|bestdeals.com|          37|
|  Shirt|    eshop.com|          58|
|    Toy|bestdeals.com|          29|
|    Toy|shopworld.com|          35|
|    Toy|  buyeasy.com|          44|
|    Toy|    eshop.com|          47|
+-------+-------------+------------+



In [26]:
#  total unique visitors
total_unique_visitors = df.select(countDistinct('CustomerID').alias('TotalUniqueVisitors')).collect()[0][0]
print(f'Total Unique Visitors: {total_unique_visitors}')

# insight  visitor demographics
visitor_demographics = df.groupBy('Country', 'Membership').agg(countDistinct('CustomerID').alias('UniqueVisitors')).sort("Country",ascending=True)
visitor_demographics.show()


Total Unique Visitors: 560
+---------+----------+--------------+
|  Country|Membership|UniqueVisitors|
+---------+----------+--------------+
|Australia|       VIP|            34|
|Australia|   Premium|            37|
|Australia|  Standard|            44|
|   Canada|   Premium|            39|
|   Canada|       VIP|            30|
|   Canada|  Standard|            35|
|  Germany|       VIP|            40|
|  Germany|   Premium|            38|
|  Germany|  Standard|            32|
|       UK|       VIP|            36|
|       UK|  Standard|            40|
|       UK|   Premium|            36|
|      USA|  Standard|            40|
|      USA|   Premium|            36|
|      USA|       VIP|            43|
+---------+----------+--------------+



In [36]:
from pyspark.sql.functions import * 

#  top-performing products based on total revenue
top_products = df.groupBy('Product').agg(round(sum('TotalSpent'),2).alias('TotalRevenue')).sort("TotalRevenue",ascending=False)
top_products.show()


+-------+------------+
|Product|TotalRevenue|
+-------+------------+
|   Book|  1174889.52|
| Laptop|  1003547.72|
|  Shirt|   944318.44|
|    Toy|   923816.41|
|   Bike|   895996.06|
+-------+------------+



In [40]:

# Define the observation period (e.g., one year from the latest order date)
observation_period_end = df.agg({"OrderDate": "max"}).collect()[0][0]

# Calculate retention based on 'CustomerID'
df = df.withColumn('Retention', col('OrderDate') >= (lit(observation_period_end) - expr('INTERVAL 1 YEAR')))

# Calculate retention rate for each customer
retention_rate = df.groupBy('CustomerID').agg((count(when(col('Retention') == True, 1)) / count('*')).alias('RetentionRate'))

# Calculate churn rate for each customer
churn_rate = df.groupBy('CustomerID').agg((count(when(col('Retention') == False, 1)) / count('*')).alias('ChurnRate'))

# Merge the retention rate and churn rate back to the original dataframe
df = df.join(retention_rate, on='CustomerID', how='left')
df = df.join(churn_rate, on='CustomerID', how='left')

# Show the updated DataFrame
df=df.sort("CustomerID",asending=True)
df.show()

2024-08-20 10:00:11,545 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1455.8 KiB
2024-08-20 10:01:37,557 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 4.4 MiB
2024-08-20 10:01:47,411 ERROR executor.Executor: Exception in task 0.0 in stage 316.0 (TID 17760)
java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.io.ReadAheadInputStream.<init>(ReadAheadInputStream.java:105)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:74)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter$SpillableIterator.spill(UnsafeExternalSorter.java:593)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.spill(UnsafeExternalSorter.jav

Py4JJavaError: An error occurred while calling o768.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 316.0 failed 1 times, most recent failure: Lost task 0.0 in stage 316.0 (TID 17760) (gateway.cloudloka.com executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.io.ReadAheadInputStream.<init>(ReadAheadInputStream.java:105)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:74)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter$SpillableIterator.spill(UnsafeExternalSorter.java:593)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.spill(UnsafeExternalSorter.java:203)
	at org.apache.spark.mem